网络流课件P10给出了最大流问题的线性规划建模,请将其输入至COPT求解器中,求解P10页中给的例子,验证求解结果与课件给出的最优流量与流值是否一致.

初始化

In [136]:
import numpy as np
import coptpy


class Node:
    def __init__(self, name: str, enter: list = None, out: list = None) -> None:
        self.name = name
        self.enter = enter if enter is not None else []
        self.out = out if out is not None else []


class Edge:
    def __init__(self, start: Node, end: Node, capacity: int) -> None:
        self.name = f"e_{start.name}{end.name}"
        self.start = start
        self.end = end
        self.capacity = capacity
        self.var = None


def init_edges(model, nodes: list, graph: np.ndarray):
    Es = [
        Edge(nodes[i], nodes[j], graph[i][j])
        for i in range(graph.shape[0])
        for j in range(graph.shape[1])
        if graph[i][j] > 0
    ]
    for e in Es:
        e.var = model.addVar(ub=e.capacity, name=e.name)
        e.start.out.append(e)
        e.end.enter.append(e)
    return Es

In [137]:
env = coptpy.Envr()
model = env.createModel("lp_example")

nodes = [Node("s"), Node("2"), Node("3"), Node("4"), Node("5"), Node("t")]

graph = np.array(
    [
        [0, 10, 10, 0, 0, 0],
        [0, 0, 0, 9, 0, 0],
        [0, 2, 0, 8, 4, 0],
        [0, 0, 0, 0, 6, 10],
        [0, 0, 0, 0, 0, 10],
        [0, 0, 0, 0, 0, 0],
    ]
)

edges = init_edges(model, nodes, graph)

Cardinal Optimizer v7.1.1. Build date Mar  4 2024
Copyright Cardinal Operations 2024. All Rights Reserved



边交叉校验

In [138]:
for edge in edges:
    print(edge.name, edge.start.name, edge.end.name, edge.capacity)

e_s2 s 2 10
e_s3 s 3 10
e_24 2 4 9
e_32 3 2 2
e_34 3 4 8
e_35 3 5 4
e_45 4 5 6
e_4t 4 t 10
e_5t 5 t 10


节点交叉校验

In [139]:
for node in nodes:
    print(node.name, [e.name for e in node.enter], [e.name for e in node.out])

s [] ['e_s2', 'e_s3']
2 ['e_s2', 'e_32'] ['e_24']
3 ['e_s3'] ['e_32', 'e_34', 'e_35']
4 ['e_24', 'e_34'] ['e_45', 'e_4t']
5 ['e_35', 'e_45'] ['e_5t']
t ['e_4t', 'e_5t'] []


约束条件添加

In [140]:
[
    model.addConstr(sum(e.var for e in node.enter) == sum(e.var for e in node.out))
    for node in nodes
    if node.enter and node.out
]

[<coptpy.Constraint: >,
 <coptpy.Constraint: >,
 <coptpy.Constraint: >,
 <coptpy.Constraint: >]

目标函数设置并求解

In [141]:
model.setObjective(
    sum([e.var for e in nodes[0].out]) - sum([e.var for e in nodes[0].enter]),
    sense=coptpy.COPT.MAXIMIZE,
)

In [142]:
model.solve()

Model fingerprint: e84f099e

Using Cardinal Optimizer v7.1.1 on Windows
Hardware has 12 cores and 16 threads. Using instruction set X86_AVX2 (10)
Maximizing an LP problem

The original problem has:
    4 rows, 9 columns and 14 non-zero elements
The presolved problem has:
    4 rows, 9 columns and 14 non-zero elements

Starting the simplex solver using up to 8 threads

Method   Iteration           Objective  Primal.NInf   Dual.NInf        Time
Dual             0    1.9102837777e+01            2           0       0.01s
Dual             5    1.8999566884e+01            0           0       0.01s

Solving finished
Status: Optimal  Objective: 1.9000000000e+01  Iterations: 5  Time: 0.01s


结果对照检验

In [145]:
for i in range(0, len(edges)):
    print(
        model.getVar(i).name,
        "=",
        model.getVar(i).getInfo("Value"),
        "/",
        model.getVar(i).getInfo("UB"),
    )
print("max =", model.objval)

e_s2 = 9.0 / 10.0
e_s3 = 10.0 / 10.0
e_24 = 9.0 / 9.0
e_32 = 0.0 / 2.0
e_34 = 6.0 / 8.0
e_35 = 4.0 / 4.0
e_45 = 5.0 / 6.0
e_4t = 10.0 / 10.0
e_5t = 9.0 / 10.0
max = 19.0
